In [ ]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
from matplotlib.gridspec import GridSpec
from matplotlib.lines import Line2D
import xarray as xr
import os; import time
import pickle
import h5py

In [ ]:
#MAIN DIRECTORIES
mainDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
scratchDirectory='/home/air673/koa_scratch/'
codeDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Project_Algorithms/Entrainment'

In [ ]:
#LOADING DATA
def GetDataDirectories(simulationNumber):
    if simulationNumber == 1:
        Directory=os.path.join(mainDirectory,'Model/cm1r20.3/run')
        res='1km'; t_res='5min'; Np_str='1e6'; Nz_str='34'
    elif simulationNumber == 2:
        Directory=scratchDirectory
        res='1km'; t_res='1min'; Np_str='50e6'; Nz_str='95'
    elif simulationNumber == 3:
        Directory=scratchDirectory
        res='250m'; t_res='1min'; Np_str='50e6'; Nz_str='95'
        
    dataDirectory = os.path.join(Directory, f"cm1out_{res}_{t_res}_{Nz_str}nz.nc")
    parcelDirectory = os.path.join(Directory,f"cm1out_pdata_{res}_{t_res}_{Np_str}np.nc")
    return dataDirectory, parcelDirectory, res,t_res,Np_str,Nz_str
    
def GetData(dataDirectory, parcelDirectory):
    dataNC = xr.open_dataset(dataDirectory, decode_timedelta=True) 
    parcelNC = xr.open_dataset(parcelDirectory, decode_timedelta=True) 
    return dataNC,parcelNC

def SubsetDataVars(dataNC):
    varList = ["thflux", "qvflux", "tsk", "cape", 
               "cin", "lcl", "lfc", "th",
               "prs", "rho", "qv", "qc",
               "qr", "qi", "qs","qg", 
               "buoyancy", "uinterp", "vinterp", "winterp",]
    
    varList += ["ptb_hadv", "ptb_vadv", "ptb_hidiff", "ptb_vidiff",
                "ptb_hturb", "ptb_vturb", "ptb_mp", "ptb_rdamp", 
                "ptb_rad", "ptb_div", "ptb_diss",]
    
    varList += ["qvb_hadv", "qvb_vadv", "qvb_hidiff", "qvb_vidiff", 
                "qvb_hturb", "qvb_vturb", "qvb_mp",]
    
    varList += ["wb_hadv", "wb_vadv", "wb_hidiff", "wb_vidiff",
                "wb_hturb", "wb_vturb", "wb_pgrad", "wb_rdamp", "wb_buoy",]

    return dataNC[varList]

[dataDirectory,parcelDirectory, res,t_res,Np_str,Nz_str] = GetDataDirectories(simulationNumber=1)
[data,parcel] = GetData(dataDirectory, parcelDirectory)

In [ ]:
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'

In [ ]:
#########################################

In [ ]:
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
path=dir2+'../Functions/'
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions


# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(PlottingFunctions, inspect.isfunction)]
# functions

In [ ]:
########################################################
#PLOTTING
plotting=True

In [ ]:
# #DOMAIN SUBSETTING
# ocean_percent=2/8

# left_to_coast=data['xh'][0]+(data['xh'][-1]-data['xh'][0])*ocean_percent
# where_coast_xh=np.where(data['xh']>=left_to_coast)[0][0]#-25
# where_coast_xf=np.where(data['xf']>=left_to_coast)[0][0]#-25
# end_xh=len(data['xh'])-1-50
# end_xf=len(data['xf'])-1-50

# print(f'x in {0}:{where_coast_xh-1} FOR SEA')
# print(f'x in {where_coast_xh}:{end_xh} FOR LAND')
# # t_end=78 
# # if res=='250m':t_end=410
# # print(f't in {0}:{t_end} (6.5 hours)')
# t_start=36 
# print(f't in {t_start}:end (8 hours)')

# profile_array_e_g=profile_array_e_g[slice(0,78+1),:,:,slice(where_coast_xh,end_xh+1)]
# profile_array_d_g=profile_array_d_g[slice(0,78+1),:,:,slice(where_coast_xh,end_xh+1)]
# profile_array_e_c=profile_array_e_c[slice(0,78+1),:,:,slice(where_coast_xh,end_xh+1)]
# profile_array_d_c=profile_array_d_c[slice(0,78+1),:,:,slice(where_coast_xh,end_xh+1)]

In [ ]:
#constants
Cp=1004 #Jkg-1K-1
Cv=717 #Jkg-1K-1
Rd=Cp-Cv #Jkg-1K-1
eps=0.608

Lx=(data['xf'][-1].item()-data['xf'][0].item())*1000 #x length (m)
Ly=(data['yf'][-1].item()-data['yf'][0].item())*1000 #y length (m)
Np=len(parcel['xh']) #number of lagrangian parcles
dt=(data['time'][1]-data['time'][0]).item()/1e9 #sec
dx=(data['xf'][1].item()-data['xf'][0].item())*1e3 #meters
dy=(data['yf'][1].item()-data['yf'][0].item())*1e3 #meters
xs=data['xf'].values*1000
ys=data['yf'].values*1000
zs=data['zf'].values*1000

def zf(z):
    k=z #z is the # level of z
    out=data['zf'].values[k]*1000
    
    return out
# def rho(x,y,z,t):
#     p=data['prs'].isel(xh=x,yh=y,zh=z,time=t).item()
#     p0=101325 #Pa
#     theta=data['th'].isel(xh=x,yh=y,zh=z,time=t).item()
#     T=theta*(p/p0)**(Rd/Cp)
#     qv=data['qv'].isel(xh=x,yh=y,zh=z,time=t).item()
#     # Tv=T*(1+eps*qv)
#     Tv=T*(eps+qv)/(eps*(1+qv))
#     rho = p/(Rd*Tv)
#     out=rho
#     return out

def rho(x,y,z,rho_data_t):
    out=rho_data_t[z,y,x]
    return out
def m(t):
    rho_data_t=data['rho'].isel(time=t).data
    
    m=0
    #triple sum
    for k in range(len(data['zh'])):
        dz=(zf(k+1)-zf(k))
        for j in range(len(data['yh'])):
            for i in range(len(data['xh'])):
                rho_out=rho(i,j,k,rho_data_t)
                m+=rho_out*dz

    #triple sum
    out=m*dx*dy/Np
    return out

In [ ]:
if plotting==True:
    #Calculate Mass Constant
    # calculate='single_time'
    # calculate=True
    calculate=False
    
    if calculate==True:
        Nt=len(data['time'])
        m_arr=np.zeros((Nt))
        for t in np.arange(Nt):
            if np.mod(t,25)==0: print(t)
            m_arr[t]=m(t)
        dir3=dir+f'Project_Algorithms/Entrainment/OUTPUT/'
        np.save(dir3+f'Mass_Array_{res}_{t_res}_{Np_str}.npy', m_arr)
    elif calculate=='single_time':
        Nt=len(data['time'])
        m_arr=np.zeros((Nt))
    
        t=0 #len(data['time'])//2 #Pick some middle time
        m_300=m(t)
        for t in np.arange(Nt):
            m_arr[t]=m_300 #UNCOMMENT FOR FULL CALCULATION
        dir3=dir+f'Project_Algorithms/Entrainment/OUTPUT/'
        np.save(dir3+f'Mass_Array_{res}_{t_res}_{Np_str}.npy', m_arr)
    else:
        dir3=dir+f'Project_Algorithms/Entrainment/OUTPUT/'
        m_arr = np.load(dir3+f'Mass_Array_{res}_{t_res}_{Np_str}.npy')
    
    # # TESTING
    # lst=[]
    # for t in np.arange(133):
    #     lst.append(m_arr[t])
    
    # plt.plot(lst)
    # (np.max(lst)-np.min(lst))*100/np.mean(lst)

In [ ]:
# # NONOPTIMIZED LOADING AND AVERAGING (NOT RECOMMENDED)

# PROCESSING=False
# PROCESSING=True

# if PROCESSING==False:
#     dir3=dir+f'Project_Algorithms/Entrainment/3D_entrainmentdetrainment_profiles_{res}_{t_res}_{Np_str}.h5'
# if PROCESSING==True:
#     dir3=dir+f'Project_Algorithms/Entrainment/3D_entrainmentdetrainment_profiles_PREPROCESSING_{res}_{t_res}_{Np_str}.h5'
# with h5py.File(dir3, "r") as h5f:
#     profile_array_e_g = h5f["profile_array_e_g"][:]
#     profile_array_e_c = h5f["profile_array_e_c"][:]
#     profile_array_d_g = h5f["profile_array_d_g"][:]
#     profile_array_d_c = h5f["profile_array_d_c"][:]

# def apply_constant(profile_array,apply):
#     if apply==True:
#         Nt=profile_array.shape[0]
#         Nz=profile_array.shape[1]
    
#         profile_array/=(dx*dy*dt)
#         for t in np.arange(Nt):
#             profile_array[t]*=m_arr[t]
#         for z in np.arange(Nz):
#             dz=zf(z+1)-zf(z)
#             profile_array[:,z]/=dz
#     return profile_array

# #APPLY CONSTANTS TO ENTRAINMENT VALUE
# ##################################################
# profile_array_e_g=apply_constant(profile_array_e_g,apply=True)
# profile_array_e_c=apply_constant(profile_array_e_c,apply=True)
# profile_array_d_g=-apply_constant(profile_array_d_g,apply=True)
# profile_array_d_c=-apply_constant(profile_array_d_c,apply=True)
# ##################################################

# # type='general'
# type='cloudy'

# if type=='general':
#     profile_array_e=profile_array_e_g
#     profile_array_d=profile_array_d_g
#     profile_array_net=profile_array_e-profile_array_d
# if type=='cloudy':
#     profile_array_e=profile_array_e_c
#     profile_array_d=profile_array_d_c
#     profile_array_net=profile_array_e-profile_array_d

In [ ]:
# #OPTIMIZED LOADING AND AVERAGING
# def apply_constant_tbyt(profile_array,t,apply):
#     if apply==True:
#         Nt=len(data['time'])
#         Nz=len(data['zh'])
    
#         profile_array/=(dx*dy*dt)
#         profile_array*=m_arr[t]
#         for z in np.arange(Nz):
#             dz=zf(z+1)-zf(z)
#             profile_array[z]/=dz
#     return profile_array


# PROCESSING=False
# PROCESSING=True

# if PROCESSING==False:
#     dir3=dir+f'Project_Algorithms/Entrainment/3D_entrainmentdetrainment_profiles_{res}_{t_res}_{Np_str}.h5'
# if PROCESSING==True:
#     dir3=dir+f'Project_Algorithms/Entrainment/3D_entrainmentdetrainment_profiles_PREPROCESSING_{res}_{t_res}_{Np_str}.h5'

# def load_get_mean(e_string,d_string,type):
#     Nt=len(data['time']); Nz=len(data['zh']); 
#     e_output_array = np.zeros((Nt, Nz))
#     d_output_array = np.zeros((Nt, Nz))
#     net_output_array = np.zeros((Nt, Nz))

    
#     with h5py.File(dir3, "r") as h5f:
#         #Reading

#         for t in np.arange(Nt):
#             print(t)
#             profile_array_e = h5f[e_string][t]
#             profile_array_d = h5f[d_string][t]
    
#             #Applying Constants
#             profile_array_e=apply_constant_tbyt(profile_array_e,t,apply=True)
#             profile_array_d=-apply_constant_tbyt(profile_array_d,t,apply=True)
    
#             profile_array_net=profile_array_e-profile_array_d

#             e_mean_yx=np.mean(profile_array_e, axis = (1,2))
#             d_mean_yx=np.mean(profile_array_d, axis = (1,2))
#             net_mean_yx=np.mean(profile_array_net, axis = (1,2))

#             e_output_array[t]=e_mean_yx
#             d_output_array[t]=d_mean_yx
#             net_output_array[t]=net_mean_yx

    
#     return e_output_array, d_output_array, net_output_array

# # #TESTING
# # test=np.random.random((2,4,4,4))
# # one=np.mean(test[0],axis=(1,2))
# # two=np.mean(test[1],axis=(1,2))
# # full=np.mean(test,axis=(2,3))
# # print(full[0]==one)
# # print(full[1]==two)

# type='general'
# type='cloudy'

# if type=='general':
#     e_string="profile_array_e_g"
#     d_string="profile_array_d_g"
#     [profile_array_e,profile_array_d,profile_array_net] = load_get_mean(e_string,d_string,type)
    
# if type=='cloudy':
#     e_string="profile_array_e_c"
#     d_string="profile_array_d_c"
#     [profile_array_e,profile_array_d,profile_array_net] = load_get_mean(e_string,d_string,type)

In [ ]:
PROCESSING=False
PROCESSING=True

if PROCESSING==False:
    dir3=dir+f'Project_Algorithms/Entrainment/OUTPUT/3D_entrainmentdetrainment_profiles_{res}_{t_res}_{Np_str}.h5'
if PROCESSING==True:
    dir3=dir+f'Project_Algorithms/Entrainment/OUTPUT/3D_entrainmentdetrainment_profiles_PREPROCESSING_{res}_{t_res}_{Np_str}.h5'

import dask
import dask.array as da
from dask.diagnostics import ProgressBar

# Open the dataset with chunking
ds = xr.open_dataset(
    dir3,
    engine='h5netcdf',  # Use the correct engine based on the file type
    phony_dims='sort',
    chunks = {'phony_dim_0': 100,
              'phony_dim_1': 34,
              'phony_dim_2': 100,
              'phony_dim_3': 64#128
}
)

# Rename the dimensions
ds = ds.rename({
    'phony_dim_0': 't',   # Rename phony_dim_0 to 't'
    'phony_dim_1': 'z',   # Rename phony_dim_1 to 'z'
    'phony_dim_2': 'y',   # Rename phony_dim_2 to 'y'
    'phony_dim_3': 'x'    # Rename phony_dim_3 to 'x'
})

In [ ]:
#EVEN MORE OPTIMIZED WITH DASK

def apply_constant(profile_array, apply, dx, dy, dt, m_arr, dz):
    if apply:
        # Step 1: Divide by dx * dy * dt (scalar)
        profile_array = profile_array / (dx * dy * dt)

        # Step 2: Multiply by m_arr[t] — shape (Nt,) needs to broadcast to (Nt, Nz, Ny, Nx)
        m_arr_broadcasted = da.asarray(m_arr)[:, None, None, None]  # Shape: (Nt, 1, 1, 1)
        profile_array = profile_array * m_arr_broadcasted

        # Step 3: Divide by dz[z] — zf of shape (Nz + 1,), so dz is (Nz,)
        dz = da.asarray(dz)  # Shape: (Nz,)
        dz_broadcasted = dz[None, :, None, None]  # Shape: (1, Nz, 1, 1)
        profile_array = profile_array / dz_broadcasted

    return profile_array

profile_vars = ['e_c', 'd_c', 'e_g', 'd_g']
tz_profiles = {}
zf2=data['zf'].data*1000
dz=zf2[1:]-zf2[:-1]

with ProgressBar():
    for var in profile_vars:
        profile_array = ds[f'profile_array_{var}']
        # Apply the same transformation to each variable
        profile_array_updated = apply_constant(profile_array, apply=True, dx=dx, dy=dy, dt=dt, m_arr=m_arr, dz=dz)
        # Compute the zx mean
        tz_profile = profile_array_updated.mean(dim=('y', 'x'))
        tz_profiles[f'tz_profile_array_{var}'] = tz_profile#.compute()

    # Persist the results (keep them in memory across workers)
    tz_profiles_persisted = {k: v.persist() for k, v in tz_profiles.items()}
 
    # Compute all profiles at once to avoid individual .compute() calls
    computed_profiles = dask.compute(*tz_profiles_persisted.values())

    # Map the computed results back to the dictionary
    tz_profiles = dict(zip(tz_profiles_persisted.keys(), computed_profiles))


In [ ]:
#SAVING
dir3=dir2+f'Project_Algorithms/Entrainment/OUTPUT/Entrainment_tz_profiles_{res}_{t_res}_{Np_str}.npz'
np.savez(dir3, **tz_profiles)

In [ ]:
########################################################################

In [ ]:
#LOADING
dir3=dir2+f'Project_Algorithms/Entrainment/OUTPUT/Entrainment_tz_profiles_{res}_{t_res}_{Np_str}.npz'
tz_profiles=np.load(dir3)

In [ ]:
type1='general'
type1='cloudy'
profile_array_e = tz_profiles['tz_profile_array_e_'+type1[0]].copy()
profile_array_d = -tz_profiles['tz_profile_array_d_'+type1[0]].copy()
profile_array_net=profile_array_e-profile_array_d

In [ ]:
# #OLD PLOTTING FUNCTION

# #Plotting
# ############################################################
# import matplotlib.pyplot as plt
# from matplotlib.gridspec import GridSpec
# import numpy as np

# fig = plt.figure(figsize=(10, 8))
# gs = GridSpec(2, 2, figure=fig)

# ######
# cmap1 = plt.cm.viridis
# cmap2 = plt.cm.seismic 
# n_levels=29
# ######

# ######
# vmax_shared = np.max([np.max(profile_array_e), np.max(profile_array_d)])
# print(np.max([np.max(profile_array_e), np.max(profile_array_d)]))
# norm_shared = mcolors.Normalize(vmin=0, vmax=vmax_shared)
# ######

# # First subplot: Entrainment
# ########################################
# ax1 = fig.add_subplot(gs[0, 0])
# # contour1 = ax1.contourf(profile_array_e.T, cmap=cmap1)
# contour1 = ax1.contourf(profile_array_e.T, cmap=cmap1, norm=norm_shared, levels=n_levels)
# cbar1=fig.colorbar(contour1, ax=ax1)
# Nz = len(data['zh'])
# ax1.set_yticks(np.arange(Nz))
# new_ytick_labels = np.round(data['zf'].values[:Nz], 2)
# ax1.set_yticklabels(new_ytick_labels, fontsize=8, rotation=0)
# ax1.set_ylabel('z (km)');ax1.set_xlabel('t (timesteps)')
# ax1.set_title('Entrainment using Lagrangian Binary Array',fontsize=8)

# # Second subplot: Detrainment
# ########################################
# ax2 = fig.add_subplot(gs[0, 1])
# # contour2 = ax2.contourf(profile_array_d.T, cmap=cmap1)
# contour2 = ax2.contourf(profile_array_d.T, cmap=cmap1, norm=norm_shared, levels=n_levels)
# cbar2 = fig.colorbar(contour2, ax=ax2)
# ax2.set_yticks(np.arange(Nz))
# new_ytick_labels = np.round(data['zf'].values[:Nz], 2)
# ax2.set_yticklabels(new_ytick_labels, fontsize=8, rotation=0)
# ax2.set_ylabel('z (km)');ax2.set_xlabel('t (timesteps)')
# ax2.set_title('Detrainment')

# # Third subplot: Net Entrainment
# ########################################


# # #OLD METHOD, DOESNT BALANCE COLOR LEVELS
# # # Normalize with a balanced vmin and vmax
# # levels=49; vmin=np.min(profile_array_net);vmax=np.max(profile_array_net)
# # # vmin=-np.max(abs(profile_array_net)); vmax=+np.max(abs(profile_array_net))
# # norm = mcolors.TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)

# # Normalize with a balanced vmin and vmax
# vmin=-np.max(abs(profile_array_net)); vmax=+np.max(abs(profile_array_net))
# levels = np.linspace(vmin, vmax, n_levels)
# norm = mcolors.BoundaryNorm(boundaries=levels, ncolors=256)
# cmap = plt.get_cmap('RdBu_r', n_levels)

# ax3 = fig.add_subplot(gs[1, 0])
# contour3 = ax3.contourf((profile_array_net).T, cmap=cmap2, norm=norm, levels=levels)
# # contour3 = ax3.contourf((profile_array_net).T, cmap=cmap2, levels=30,vmin=-np.max(abs(profile_array_net)), vmax=+np.max(abs(profile_array_net)))
# # cmap2 = plt.get_cmap('RdBu', 29);contour3 = ax3.pcolor(profile_array_net.T, cmap=cmap2, norm=norm, shading='auto')
# cbar3 = fig.colorbar(contour3, ax=ax3, norm=norm)

# #FIXING TICKS
# ax3.set_yticks(np.arange(Nz))
# new_ytick_labels = np.round(data['zf'].values[:Nz], 2)
# ax3.set_yticklabels(new_ytick_labels, fontsize=8, rotation=0)
# ax3.set_ylabel('z (km)');ax3.set_xlabel('t (timesteps)')
# ax3.set_title('Entrainment - Detrainment')

# #FIXING SCIENTIFIC NOTATION

# def apply_scientific_notation_colorbar(cbars):
#     from matplotlib.ticker import ScalarFormatter
#     formatter = ScalarFormatter(useMathText=True)
#     formatter.set_powerlimits((-2, 2))  # Adjust the range for scientific notation
#     for cbar in cbars:  # These must be Colorbar instances
#         cbar.formatter = formatter
#         cbar.update_ticks()
# apply_scientific_notation_colorbar([cbar1,cbar2,cbar3])

# # Display the plot
# plt.tight_layout()

# #TESTING
# print(f"Max of profile_array_e: {np.max(profile_array_e)}")
# print(f"Max of profile_array_d: {np.max(profile_array_d)}")

# ###################### FIXING Y TICKS
# Nz = len(data['zh'])
# step = 4  # change to 2, 5, etc. depending on how spaced you want them
# ytick_pos = np.arange(0, Nz, step)
# ytick_labels = np.round(data['zf'].values[ytick_pos], 2)
# for axis in [ax1,ax2,ax3]:
#     axis.set_yticks(ytick_pos)
#     axis.set_yticklabels(ytick_labels, fontsize=8, rotation=0)

In [ ]:
#NEW PLOTTING METHOD
if plotting==True:
    
    ######
    cmap1 = plt.cm.viridis
    cmap2 = plt.cm.seismic 
    n_levels=29
    ######
    
    ######
    vmax_shared = np.max([np.max(profile_array_e), np.max(profile_array_d)])
    norm_shared = mcolors.Normalize(vmin=0, vmax=vmax_shared)
    norm_shared = None #COMMENT OUT IF COLORBARS SHOULD BE SHARED
    ######
    
    # === Create figure and subplots ===
    # fig, axs = plt.subplots(2, 2, figsize=(15, 10))
    fig = plt.figure(figsize=(10, 8))
    from matplotlib.gridspec import GridSpec
    gs = GridSpec(2, 2, figure=fig)
    ax1 = fig.add_subplot(gs[0, 0])
    ax2 = fig.add_subplot(gs[0, 1])
    ax3 = fig.add_subplot(gs[1, 0])
    
    # === Base Plot configuration parameters ===
    plot_kwargs = {
        'PlotData': None, #THIS MUST BE SET SOMEWHERE
        'xTickLabels': None, 'yTickLabels': None, #THESE MUST BE SET SOMEWHERE
        'contour_type': 'fill',
        'num_xticks': 10,'round_xticks': 0, 'xTickInterval': 100,
        'num_yticks': 15,'round_yticks': 2, 'yTickInterval': None,
        'add_colorbar': True,'fig': fig, 'levels': 29, 'colorbar_label_rotation': 0, 'colorbar_label': None,
        'xlabel': "t (timesteps)", 'ylabel': "z (km)",
        'solid_contour_labels': True, 'solid_contour_round': None,
        'xtick_rotation': 0, 'ytick_rotation': 0, 'cbar_rotation': 0,
        'save_path': None, 'save_dpi': 300,
        'colorbar_kwargs': {
                'extend': 'both'
            },
    
        'norm': norm_shared
    }
    
    # === Plot 1 ===
    plot_data1 = profile_array_e.copy().T
    # plot_data1[plot_data1==0]=np.nan
    y = data['zh'].data  # len 95
    x = np.arange(profile_array_e.shape[0])  # len 661
    plot_kwargs['xTickLabels'] = x
    plot_kwargs['yTickLabels'] = y
    
    plot_kwargs1 = plot_kwargs.copy()
    plot_kwargs1['PlotData'] = plot_data1
    plot_kwargs1['cmap'] = cmap1
    [contour1,cbar1]=UltimateContourPlot(ax1, **plot_kwargs1)
    ax1.set_ylim(0,19)
    ax1.set_title('Entrainment')
    
    # # === Plot 2 ===
    plot_data2 = profile_array_d.copy().T
    # plot_data2[plot_data2==0]=np.nan
    plot_kwargs2 = plot_kwargs.copy()
    plot_kwargs2['PlotData'] = plot_data2
    plot_kwargs2['cmap'] = cmap1
    [contour2,cbar2]=UltimateContourPlot(ax2, **plot_kwargs2)
    ax2.set_ylim(0,19)
    ax2.set_title('Detrainment')
    
    # # === Plot 3 ===
    plot_data3 = profile_array_net.copy().T
    #######################################
    vmin=-np.max(abs(profile_array_net))/2; vmax=+np.max(abs(profile_array_net))
    percentile_vminmax=False
    if percentile_vminmax==True:
        ####
        vmin = np.percentile(profile_array_net[profile_array_net<0], 1)
        vmax = np.percentile(profile_array_net[profile_array_net>0], 99)
        ####    
    levels = np.linspace(vmin, vmax, n_levels)
    norm = mcolors.BoundaryNorm(boundaries=levels, ncolors=256)
    #######################################
    
    # plot_data3[plot_data3==0]=np.nan
    plot_kwargs3 = plot_kwargs.copy()
    plot_kwargs3['PlotData'] = plot_data3
    plot_kwargs3['cmap'] = cmap2
    plot_kwargs3['norm'] = norm
    plot_kwargs3['levels'] = levels
    [contour3,cbar3]=UltimateContourPlot(ax3, **plot_kwargs3)
    ax3.set_ylim(0,19)
    ax3.set_title('Net Entrainment')
    
    
    
    ################################################################################
    
    #APPLY SCIENTIFIC NOTATION
    def apply_scientific_notation_colorbar(cbars):
        from matplotlib.ticker import ScalarFormatter
        formatter = ScalarFormatter(useMathText=True)
        formatter.set_powerlimits((-2, 2))  # Adjust the range for scientific notation
        for cbar in cbars:  # These must be Colorbar instances
            cbar.formatter = formatter
            cbar.update_ticks()
    apply_scientific_notation_colorbar([cbar1,cbar2,cbar3])

    #TIGHT PLOTTING LAYOUT
    fig.tight_layout()

In [ ]:
e=np.mean(profile_array_e,axis=(0))
d=np.mean(profile_array_d,axis=(0))
net=np.mean(profile_array_net,axis=(0))

plt.plot(e,data['zh'],color='blue',label='entrainment')
plt.plot(d,data['zh'],color='red',label='detrainment')
plt.plot(net,data['zh'],linestyle='dashed',color='black',label='entrainment - detrainment')
plt.axvline(0,color='black')

plt.legend(); plt.title('2D Entrainment and Detrainment Using Lagrangian Binary Array')

from matplotlib.ticker import ScalarFormatter
formatter = ScalarFormatter(useMathText=True)
formatter.set_scientific(True)
formatter.set_powerlimits((-1, 1))
plt.gca().xaxis.set_major_formatter(formatter)

In [ ]:
##################################
#Z-X Plot

In [ ]:
import xarray as xr
import dask.array as da

def apply_constant(profile_array, apply, dx, dy, dt, m_arr, dz):
    if apply:
        # Step 1: Divide by dx * dy * dt (scalar)
        profile_array = profile_array / (dx * dy * dt)

        # Step 2: Multiply by m_arr[t] — shape (Nt,) needs to broadcast to (Nt, Nz, Ny, Nx)
        m_arr_broadcasted = da.asarray(m_arr)[:, None, None, None]  # Shape: (Nt, 1, 1, 1)
        profile_array = profile_array * m_arr_broadcasted

        # Step 3: Divide by dz[z] — zf of shape (Nz + 1,), so dz is (Nz,)
        dz = da.asarray(dz)  # Shape: (Nz,)
        dz_broadcasted = dz[None, :, None, None]  # Shape: (1, Nz, 1, 1)
        profile_array = profile_array / dz_broadcasted

    return profile_array

profile_vars = ['e_c', 'd_c', 'e_g', 'd_g']
zx_profiles = {}
zf2=data['zf'].data*1000
dz=zf2[1:]-zf2[:-1]

with ProgressBar():
    for var in profile_vars:
        profile_array = ds[f'profile_array_{var}']
        # Apply the same transformation to each variable
        profile_array_updated = apply_constant(profile_array, apply=True, dx=dx, dy=dy, dt=dt, m_arr=m_arr, dz=dz)
        # Compute the zx mean
        zx_profile = profile_array_updated.mean(dim=('t', 'y'))
        zx_profiles[f'zx_profile_array_{var}'] = zx_profile#.compute()

    # Persist the results (keep them in memory across workers)
    zx_profiles_persisted = {k: v.persist() for k, v in zx_profiles.items()}
 
    # Compute all profiles at once to avoid individual .compute() calls
    computed_profiles = dask.compute(*zx_profiles_persisted.values())

    # Map the computed results back to the dictionary
    tz_profiles = dict(zip(zx_profiles_persisted.keys(), computed_profiles))

In [ ]:
#SAVING
dir3=dir2+f'Project_Algorithms/Entrainment/Entrainment_zx_profiles_{res}_{t_res}_{Np_str}.npz'
np.savez(dir3, **tz_profiles)

In [ ]:
##########################################

In [ ]:
#LOADING
dir3=dir2+f'Project_Algorithms/Entrainment/Entrainment_zx_profiles_{res}_{t_res}_{Np_str}.npz'
zx_profiles=np.load(dir3)

In [ ]:
type1='general'
type1='cloudy'
profile_array_e = zx_profiles['zx_profile_array_e_'+type1[0]].copy()
profile_array_d = -zx_profiles['zx_profile_array_d_'+type1[0]].copy()
profile_array_net=profile_array_e-profile_array_d

In [ ]:
#Entrainment

#Plotting
############################################################
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import numpy as np

fig = plt.figure(figsize=(10, 8))
gs = GridSpec(2, 2, figure=fig)

######
cmap1 = plt.cm.viridis
cmap2 = plt.cm.seismic 
n_levels=29
######

######
vmax_shared = np.max([np.max(profile_array_e), np.max(profile_array_d)])
print(np.max([np.max(profile_array_e), np.max(profile_array_d)]))
norm_shared = mcolors.Normalize(vmin=0, vmax=vmax_shared)
######

# First subplot: Entrainment
########################################
ax1 = fig.add_subplot(gs[0, 0])
# contour1 = ax1.contourf(profile_array_e.T, cmap=cmap1)
contour1 = ax1.contour(profile_array_e, cmap=cmap1, norm=norm_shared, levels=n_levels)
cbar1=fig.colorbar(contour1, ax=ax1)
Nz = len(data['zh'])
ax1.set_yticks(np.arange(Nz))
new_ytick_labels = np.round(data['zf'].values[:Nz], 2)
ax1.set_yticklabels(new_ytick_labels, fontsize=8, rotation=0)
ax1.set_ylabel('z (km)');ax1.set_xlabel('x (km)')
ax1.set_title('Entrainment using Lagrangian Binary Array',fontsize=8)

# Second subplot: Detrainment
########################################
ax2 = fig.add_subplot(gs[0, 1])
# contour2 = ax2.contourf(profile_array_d.T, cmap=cmap1)
contour2 = ax2.contour(profile_array_d, cmap=cmap1, norm=norm_shared, levels=n_levels)
cbar2 = fig.colorbar(contour2, ax=ax2)
ax2.set_yticks(np.arange(Nz))
new_ytick_labels = np.round(data['zf'].values[:Nz], 2)
ax2.set_yticklabels(new_ytick_labels, fontsize=8, rotation=0)
ax2.set_ylabel('z (km)');ax2.set_xlabel('x (km)')
ax2.set_title('Detrainment')

# Third subplot: Net Entrainment
########################################


# #OLD METHOD, DOESNT BALANCE COLOR LEVELS
# # Normalize with a balanced vmin and vmax
# levels=49; vmin=np.min(profile_array_net);vmax=np.max(profile_array_net)
# # vmin=-np.max(abs(profile_array_net)); vmax=+np.max(abs(profile_array_net))
# norm = mcolors.TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)

# Normalize with a balanced vmin and vmax
vmin=-np.max(abs(profile_array_net)); vmax=+np.max(abs(profile_array_net))
levels = np.linspace(vmin, vmax, n_levels)
norm = mcolors.BoundaryNorm(boundaries=levels, ncolors=256)
cmap = plt.get_cmap('RdBu_r', n_levels)

ax3 = fig.add_subplot(gs[1, 0])
contour3 = ax3.contour((profile_array_net), cmap=cmap2, norm=norm, levels=levels)
# contour3 = ax3.contourf((profile_array_net).T, cmap=cmap2, levels=30,vmin=-np.max(abs(profile_array_net)), vmax=+np.max(abs(profile_array_net)))
# cmap2 = plt.get_cmap('RdBu', 29);contour3 = ax3.pcolor(profile_array_net.T, cmap=cmap2, norm=norm, shading='auto')
cbar3 = fig.colorbar(contour3, ax=ax3, norm=norm)

#FIXING YTICKS
ax3.set_yticks(np.arange(Nz))
new_ytick_labels = np.round(data['zf'].values[:Nz], 2)
ax3.set_yticklabels(new_ytick_labels, fontsize=8, rotation=0)
ax3.set_ylabel('z (km)');ax3.set_xlabel('x (km)')
ax3.set_title('Entrainment - Detrainment')

#FIXING XTICKS
fix_tick_labels([ax1,ax2,ax3], data, data_dim='x', tick_axis='x', d_xtick=85, d_ytick=20, cell_loc='center',round=1,meters=False) 


#FIXING SCIENTIFIC NOTATION

def apply_scientific_notation_colorbar(cbars):
    from matplotlib.ticker import ScalarFormatter
    formatter = ScalarFormatter(useMathText=True)
    formatter.set_powerlimits((-2, 2))  # Adjust the range for scientific notation
    for cbar in cbars:  # These must be Colorbar instances
        cbar.formatter = formatter
        cbar.update_ticks()
apply_scientific_notation_colorbar([cbar1,cbar2,cbar3])

# Display the plot
plt.tight_layout()

#TESTING
print(f"Max of profile_array_e: {np.max(profile_array_e)}")
print(f"Max of profile_array_d: {np.max(profile_array_d)}")

In [ ]:
#COMBINED ENTRAINMENT
PROCESSING=False
PROCESSING=True

# if PROCESSING==False:
#     dir3=dir+f'Project_Algorithms/Entrainment/OUTPUT/3D_entrainmentdetrainment_combined_profiles_{res}_{t_res}_{Np_str}.h5'
# if PROCESSING==True:
#     dir3=dir+f'Project_Algorithms/Entrainment/OUTPUT/3D_entrainmentdetrainment_combined_profiles_PREPROCESSING_{res}_{t_res}_{Np_str}.h5'

if PROCESSING==False:
    dir3=dir+f'Project_Algorithms/Entrainment/OUTPUT/3D_entrainmentdetrainment_profiles_{res}_{t_res}_{Np_str}.h5'
if PROCESSING==True:
    dir3=dir+f'Project_Algorithms/Entrainment/OUTPUT/3D_entrainmentdetrainment_profiles_PREPROCESSING_{res}_{t_res}_{Np_str}.h5'

import dask
import dask.array as da
from dask.diagnostics import ProgressBar

# Open the dataset with chunking
ds2 = xr.open_dataset(
    dir3,
    engine='h5netcdf',  # Use the correct engine based on the file type
    phony_dims='sort',
    chunks = {'phony_dim_0': 100,
              'phony_dim_1': 34,
              'phony_dim_2': 100,
              'phony_dim_3': 64#128
}
)

# Rename the dimensions
ds2 = ds2.rename({
    'phony_dim_0': 't',   # Rename phony_dim_0 to 't'
    'phony_dim_1': 'z',   # Rename phony_dim_1 to 'z'
    'phony_dim_2': 'y',   # Rename phony_dim_2 to 'y'
    'phony_dim_3': 'x'    # Rename phony_dim_3 to 'x'
})

In [ ]:
#EVEN MORE OPTIMIZED WITH DASK

def apply_constant(profile_array, apply, dx, dy, dt, m_arr, dz):
    if apply:
        # Step 1: Divide by dx * dy * dt (scalar)
        profile_array = profile_array / (dx * dy * dt)

        # Step 2: Multiply by m_arr[t] — shape (Nt,) needs to broadcast to (Nt, Nz, Ny, Nx)
        m_arr_broadcasted = da.asarray(m_arr)[:, None, None, None]  # Shape: (Nt, 1, 1, 1)
        profile_array = profile_array * m_arr_broadcasted

        # Step 3: Divide by dz[z] — zf of shape (Nz + 1,), so dz is (Nz,)
        dz = da.asarray(dz)  # Shape: (Nz,)
        dz_broadcasted = dz[None, :, None, None]  # Shape: (1, Nz, 1, 1)
        profile_array = profile_array / dz_broadcasted

    return profile_array

profile_vars = ['c_to_g_E', 'g_to_c_E','c_to_g_D', 'g_to_c_D']
tz_profiles = {}
zf2=data['zf'].data*1000
dz=zf2[1:]-zf2[:-1]

with ProgressBar():
    for var in profile_vars:
        profile_array = ds[f'profile_array_{var}']
        # Apply the same transformation to each variable
        profile_array_updated = apply_constant(profile_array, apply=True, dx=dx, dy=dy, dt=dt, m_arr=m_arr, dz=dz)
        # Compute the zx mean
        tz_profile = profile_array_updated.mean(dim=('y', 'x'))
        tz_profiles[f'tz_profile_array_{var}'] = tz_profile#.compute()

    # Persist the results (keep them in memory across workers)
    tz_profiles_persisted = {k: v.persist() for k, v in tz_profiles.items()}
 
    # Compute all profiles at once to avoid individual .compute() calls
    computed_profiles = dask.compute(*tz_profiles_persisted.values())

    # Map the computed results back to the dictionary
    tz_profiles = dict(zip(tz_profiles_persisted.keys(), computed_profiles))


In [ ]:
profile_array_c_to_g_E = tz_profiles['tz_profile_array_c_to_g_E'].copy()
profile_array_g_to_c_E = tz_profiles['tz_profile_array_g_to_c_E'].copy()
profile_array_c_to_g_D = tz_profiles['tz_profile_array_c_to_g_D'].copy()
profile_array_g_to_c_D = tz_profiles['tz_profile_array_g_to_c_D'].copy()

#SAVING
dir3=dir2+f'Project_Algorithms/Entrainment/OUTPUT/Combined_Entrainment_tz_profiles_{res}_{t_res}_{Np_str}.npz'
np.savez(dir3, **tz_profiles)

In [ ]:
#LOADING
dir3=dir2+f'Project_Algorithms/Entrainment/OUTPUT/Entrainment_tz_profiles_{res}_{t_res}_{Np_str}.npz'
tz_profiles=np.load(dir3)

type1='general'
profile_array_e_g = tz_profiles['tz_profile_array_e_'+type1[0]].copy()
profile_array_d_g = -tz_profiles['tz_profile_array_d_'+type1[0]].copy()

type1='cloudy'
profile_array_e_c = tz_profiles['tz_profile_array_e_'+type1[0]].copy()
profile_array_d_c = -tz_profiles['tz_profile_array_d_'+type1[0]].copy()

In [ ]:
#LOADING
dir3=dir2+f'Project_Algorithms/Entrainment/OUTPUT/Combined_Entrainment_tz_profiles_{res}_{t_res}_{Np_str}.npz'
tz_profiles=np.load(dir3)

# Extract and make independent copies of the four arrays
profile_array_c_to_g_E = tz_profiles['tz_profile_array_c_to_g_E'].copy()
profile_array_g_to_c_E = tz_profiles['tz_profile_array_g_to_c_E'].copy()
profile_array_c_to_g_D = tz_profiles['tz_profile_array_c_to_g_D'].copy()
profile_array_g_to_c_D = tz_profiles['tz_profile_array_g_to_c_D'].copy()

In [ ]:
# def plot_transfer_rate(ax, c_to_g, g_to_c,title):
#     zh=data['zh'].data

#     # Mean profiles
#     c_to_g_mean = np.mean(c_to_g, axis=0)
#     g_to_c_mean = np.mean(g_to_c, axis=0)

#     # Plot transfers
#     ax.plot(c_to_g_mean, zh, color='red', label='Cloudy → General')
#     ax.plot(g_to_c_mean, zh, color='blue', label='General → Cloudy')
#     ax.axvline(0, color='black', linewidth=1)

#     # Reference lines
#     # ax.axhline(cloudbase, color='purple', linestyle='dashed',lw=1.2)
#     # ax.axhline(MeanLFC / 1000, color='forestgreen', linestyle='dashed',lw=1.2)

#     # Labeling and formatting
#     ax.set_title(f"{title}")
#     ax.set_xlabel('Mass Transfer Rate')
#     ax.set_xlabel(r"($kg m^{-3} s^{-1}$)")  
#     ax.set_ylabel('z (km)')
#     # ax.set_xlim(left=0)
#     ax.legend()

#     apply_scientific_notation([ax])

# def plot_transfer_ratio(ax, profile_array_e_g, c_to_g, profile_array_e_c, g_to_c, title):
#     zh = data['zh'].data

#     # Compute mean profiles first
#     mean_c_to_g = np.mean(c_to_g, axis=0)
#     mean_g_to_c = np.mean(g_to_c, axis=0)
    
#     mean_e_g = np.mean(profile_array_e_g, axis=0)
#     mean_e_c = np.mean(profile_array_e_c, axis=0)
#     mean_d_g = np.mean(profile_array_d_g, axis=0)
#     mean_d_c = np.mean(profile_array_d_c, axis=0)

#     # Mask ratios where denominator is too small
#     threshold = 0
#     with np.errstate(divide='ignore', invalid='ignore'):
#         ratio_1 = np.where(mean_e_c > threshold, mean_g_to_c / mean_e_c, np.nan)
#         ratio_2 = np.where(mean_d_g > threshold, mean_g_to_c / mean_d_g, np.nan)
#         ratio_3 = np.where(mean_e_g > threshold, mean_c_to_g / mean_e_g, np.nan)
#         ratio_4 = np.where(mean_d_c > threshold, mean_c_to_g / mean_d_c, np.nan)
    
#     # Plot in specified order
#     ax.plot(ratio_1, zh, color='blue', label='General → Cloudy / Cloudy Entrainment')
#     ax.plot(ratio_2, zh, color='deepskyblue', label='General → Cloudy / General Detrainment')
#     ax.plot(ratio_3, zh, color='red', label='Cloudy → General / General Entrainment')
#     ax.plot(ratio_4, zh, color='orangered', label='Cloudy → General / Cloudy Detrainment')
    
#     ax.axvline(1, color='black', linestyle='dashed', linewidth=1)

#     # Reference horizontal lines
#     # ax.axhline(cloudbase, color='purple', linestyle='dashed', lw=1.2)
#     # ax.axhline(MeanLFC / 1000, color='green', linestyle='dashed', lw=1.2)

#     # Labels and limits
#     ax.set_title(f"{title}")
#     ax.set_xlabel('Ratio')
#     ax.set_ylabel('z (km)')
#     pad_fraction = 10
#     pad_multiplier = (100 + pad_fraction) / 100
#     ax.set_xlim(0, 1 * pad_multiplier)

#     ax.legend(fontsize=10.5-2, loc='upper right')
#     apply_scientific_notation([ax])


In [ ]:
# #VERTICAL PROFILES
# fig = plt.figure(figsize=(18, 6))
# gs = gridspec.GridSpec(1, 4, wspace=0.2)

# ax1 = fig.add_subplot(gs[0])
# ax2 = fig.add_subplot(gs[1])
# ax3 = fig.add_subplot(gs[2])
# ax4 = fig.add_subplot(gs[3])

# # Call function for each subplot
# plot_mean_entrainment(ax1, profile_array_e_g, profile_array_d_g, title='')
# plot_mean_entrainment(ax2, profile_array_e_c, profile_array_d_c, title='')
# plot_transfer_rate(ax3, profile_array_c_to_g, profile_array_g_to_c, title='')
# plot_transfer_ratio(ax4, profile_array_e_g,profile_array_c_to_g, profile_array_e_c,profile_array_g_to_c, title='')

# # #SAVING FIGURE
# # fig.savefig(f"PLOTS/Combined_Entrainment_VerticalProfiles_{res}_{t_res}_{Np_str}.jpg", dpi=300, bbox_inches='tight')

In [ ]:
def plot_mean_entrainment(ax, profile_array_e, profile_array_d, title, linestyle='solid'):
    zh=data['zh'].data

    # Compute mean profiles
    e = np.mean(profile_array_e, axis=0)
    d = np.mean(profile_array_d, axis=0)
    net = np.mean(profile_array_e - profile_array_d, axis=0)

    # Plot
    ax.plot(e, zh, linestyle=linestyle, color='blue', label='Entrainment')
    ax.plot(d, zh, linestyle=linestyle, color='red', label='Detrainment')
    ax.plot(net, zh, linestyle=linestyle, color='black', label='Net Entrainment')
    ax.axvline(0, color='black')

    ax.axhline(cloudbase, color='purple', linestyle='dashed', lw=1.2)
    ax.axhline(MeanLFC / 1000, color='green', linestyle='dashed', lw=1.2)
    
    ax.set_title(f"{title}",fontsize=10.5)
    ax.set_xlabel(r"($kg\ m^{-3}\ s^{-1}$)")  
    ax.set_ylabel('z (km)')
    ax.set_ylim(bottom=0)
    ax.legend()

    # Format x-axis in scientific notation
    apply_scientific_notation([ax])

In [ ]:
def compute_means_for_transfer(c_to_g_E, g_to_c_E, c_to_g_D, g_to_c_D,
                               profile_array_e_g, profile_array_e_c,
                               profile_array_d_g, profile_array_d_c):
    """Compute and return mean profiles for transfer rates and entrainment/detrainment arrays."""

    # Compute means of transfer rates
    c_to_g_E_mean = np.mean(c_to_g_E, axis=0)
    g_to_c_E_mean = np.mean(g_to_c_E, axis=0)
    c_to_g_D_mean = np.mean(c_to_g_D, axis=0)
    g_to_c_D_mean = np.mean(g_to_c_D, axis=0)

    # Compute means of entrainment/detrainment profiles
    mean_e_g = np.mean(profile_array_e_g, axis=0)
    mean_e_c = np.mean(profile_array_e_c, axis=0)
    mean_d_g = np.mean(profile_array_d_g, axis=0)
    mean_d_c = np.mean(profile_array_d_c, axis=0)

    return {
        'c_to_g_E_mean': c_to_g_E_mean,
        'g_to_c_E_mean': g_to_c_E_mean,
        'c_to_g_D_mean': c_to_g_D_mean,
        'g_to_c_D_mean': g_to_c_D_mean,
        'mean_e_g': mean_e_g,
        'mean_e_c': mean_e_c,
        'mean_d_g': mean_d_g,
        'mean_d_c': mean_d_c
    }

    
def plot_transfer_rate(ax, means, title):
    zh = data['zh'].data

    c_to_g_mean_E = means['c_to_g_E_mean']
    g_to_c_mean_E = means['g_to_c_E_mean']
    c_to_g_mean_D = means['c_to_g_D_mean']
    g_to_c_mean_D = means['g_to_c_D_mean']

    # ax.plot(g_to_c_mean_E, zh, color='blue', label='General → Cloudy',linestyle='solid')
    # ax.plot(c_to_g_mean_E, zh, color='red', label='Cloudy → General',linestyle='solid')
    ax.plot(g_to_c_mean_E, zh, color='blue', label='General → Cloudy (Entrainment)',linestyle='solid')
    ax.plot(c_to_g_mean_E, zh, color='red', label='Cloudy → General (Entrainment)',linestyle='solid')
    ax.plot(c_to_g_mean_D, zh, color='blue', label='Cloudy → General (Detrainment)',linestyle='dashed')
    ax.plot(g_to_c_mean_D, zh, color='red', label='General → Cloudy (Detrainment)',linestyle='dashed')
    ax.axvline(0, color='black', linewidth=1)

    ax.axhline(cloudbase, color='purple', linestyle='dashed', lw=1.2)
    ax.axhline(MeanLFC / 1000, color='forestgreen', linestyle='dashed', lw=1.2)

    ax.set_title(f"{title}")
    ax.set_xlabel('Mass Transfer Rate')
    ax.set_xlabel(r"($kg m^{-3} s^{-1}$)")  
    ax.set_ylabel('z (km)')
    ax.set_ylim(bottom=0)
    ax.legend()
    apply_scientific_notation([ax])

def plot_transfer_ratio(ax, means, title):
    zh = data['zh'].data

    mean_e_g = means['mean_e_g']
    mean_e_c = means['mean_e_c']
    mean_d_g = means['mean_d_g']
    mean_d_c = means['mean_d_c']
    
    mean_c_to_g_E = means['c_to_g_E_mean']
    mean_g_to_c_E = means['g_to_c_E_mean']
    mean_c_to_g_D = means['c_to_g_D_mean']
    mean_g_to_c_D = means['g_to_c_D_mean']

    threshold = 0
    with np.errstate(divide='ignore', invalid='ignore'):
        ratio_1 = np.where(mean_e_c > threshold, mean_g_to_c_E / mean_e_c, np.nan)
        ratio_2 = np.where(mean_d_g > threshold, mean_g_to_c_D / mean_d_g, np.nan)
        ratio_3 = np.where(mean_e_g > threshold, mean_c_to_g_E / mean_e_g, np.nan)
        ratio_4 = np.where(mean_d_c > threshold, mean_c_to_g_D / mean_d_c, np.nan)

    # print(np.nanmean(ratio_1))
    # print(np.nanmean(ratio_2))
    # print(np.nanmean(ratio_3))
    # print(np.nanmean(ratio_4))

    ax.plot(ratio_1*100, zh, color='blue', label='General → Cloudy / Cloudy Entrainment')
    ax.plot(ratio_2*100, zh, color='deepskyblue', label='General → Cloudy / General Detrainment')
    ax.plot(ratio_3*100, zh, color='red', label='Cloudy → General / General Entrainment')
    ax.plot(ratio_4*100, zh, color='orangered', label='Cloudy → General / Cloudy Detrainment')

    ax.axvline(0, color='black', linestyle='dashed', linewidth=1)
    # ax.axvline(1, color='black', linestyle='dashed', linewidth=1)
    ax.axvline(100, color='black', linestyle='dashed', linewidth=1)
    ax.axhline(cloudbase, color='purple', linestyle='dashed', lw=1.2)
    ax.axhline(MeanLFC / 1000, color='green', linestyle='dashed', lw=1.2)

    ax.set_title(f"{title}")
    # ax.set_xlabel('Ratio')
    ax.set_xlabel('%')
    ax.set_ylabel('z (km)')
    # ax.set_xlim(-0.05, 1.05)
    ax.set_xlim(-5, 105)
    ax.set_ylim(bottom=0)
    ax.legend(fontsize=10.5-3, loc='upper right')
    # apply_scientific_notation([ax])


In [ ]:
cloudbase=1.2
MeanLFC=2
# === Compute means for transfer rate and ratio plots ===
means = compute_means_for_transfer(
    c_to_g_E=profile_array_c_to_g_E,
    g_to_c_E=profile_array_g_to_c_E,
    c_to_g_D=profile_array_c_to_g_D,
    g_to_c_D=profile_array_g_to_c_D,
    profile_array_e_g=profile_array_e_g,
    profile_array_e_c=profile_array_e_c,
    profile_array_d_g=profile_array_d_g,
    profile_array_d_c=profile_array_d_c
)

# === Set up figure and subplots ===
fig = plt.figure(figsize=(18, 6))
gs = gridspec.GridSpec(1, 4, wspace=0.2)

ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1])
ax3 = fig.add_subplot(gs[2])
ax4 = fig.add_subplot(gs[3])

# === Plot each panel ===
plot_mean_entrainment(ax1, profile_array_e_g, profile_array_d_g, title='')
plot_mean_entrainment(ax2, profile_array_e_c, profile_array_d_c, title='')
plot_transfer_rate(ax3, means, title='')
plot_transfer_ratio(ax4, means, title='')

fix_x_limits([ax2, ax3])

# # === Save figure ===
# filename = "Combined_Entrainment_VerticalProfiles"
# SaveFigure(fig, filename)

In [ ]:
#TESTING

In [ ]:
# dm=m_arr[10]
# zf=data['zf'].data;zh=data['zh'].data;
# dz=(zf.copy()[1:]-zf.copy()[:-1])*1000
# V=(dx*dy*dz)
# rate=dm/(V*dt)

# #####
# Nx=len(data['xh']);Ny=len(data['yh']);Nt=len(data['time'])
# rate/=(Nx*Ny)#*Nt)
# #####

# plt.plot(rate,zh)
# plt.ylabel('z (km)');plt.xlabel('rate (minimum)');

# ax=plt.gca()
# apply_scientific_notation([ax])

In [ ]:
# plt.plot(dz,zh)
# plt.ylabel('z (km)');plt.xlabel('dz (m)');

In [ ]:
# #WHAT IS THE AVERAGE SUM?
# dir3=dir+f'Project_Algorithms/Entrainment/3D_entrainmentdetrainment_profiles_PREPROCESSING_{res}_{t_res}_{Np_str}.h5'
# e_string='profile_array_e_c'

# with h5py.File(dir3, "r") as h5f:
#     #Reading
#     profile_array_e = h5f[e_string][:]
# print(profile_array_e.max())